# 语言模型与数据集
:label:`sec_language_model`


在 :numref:`sec_text_preprocessing`, 中，我们看到了如何将文本数据映射到标记中，这些标记可以被看作是一系列离散的观察结果，例如单词或字符。
假设长度为 $T$ 的文本序列中的tokens依次为 $x_1, x_2, \ldots, x_T$. 
然后，在文本序列中，
$x_t$($1 \leq t \leq T$) 可被视为时间步长 $t$. 的观察值或标签。鉴于这样的文本顺序，
*语言模型*的目标是估计序列的联合概率

$$P(x_1, x_2, \ldots, x_T).$$

语言模型非常有用。 例如，一个理想的语言模型能够单独生成自然文本，只需一次绘制一个标记 $x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1)$.
与使用打字机的猴子不同，从这种模型中产生的所有文本都将作为自然语言传递，例如英语文本。此外，只需在前面的对话框片段上调整文本，就足以生成有意义的对话框。

显然，我们离设计这样一个系统还很远，因为它需要*理解*文本，而不仅仅是生成语法上合理的内容。
尽管如此，语言模型即使在其有限的形式下也能发挥巨大的作用。
例如，短语“识别语音”和“破坏美丽的海滩”听起来非常相似。
这会导致语音识别中的歧义，
这很容易通过一种语言模型来解决，这种语言模型拒绝将第二种翻译视为稀奇古怪。
同样，在文档摘要算法中
值得一提的是，“狗咬人”比“人咬狗”要频繁得多，或者“我想吃奶奶”是一个相当令人不安的说法，而“我想吃，奶奶”则要温和得多。


## 学习语言模型

显而易见的问题是，我们应该如何建模文档，甚至是一系列标记。
假设我们在单词级别标记文本数据。
我们可以求助于我们应用于序列模型的分析：numref:`sec_sequence`.
让我们从应用基本概率规则开始:

$$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^T P(x_t  \mid  x_1, \ldots, x_{t-1}).$$

例如，
包含四个单词的文本序列的概率如下所示:

$$P(\text{deep}, \text{learning}, \text{is}, \text{fun}) =  P(\text{deep}) P(\text{learning}  \mid  \text{deep}) P(\text{is}  \mid  \text{deep}, \text{learning}) P(\text{fun}  \mid  \text{deep}, \text{learning}, \text{is}).$$

为了计算语言模型，我们需要计算
单词的概率和给定单词的条件概率
前面的几句话。
这种可能性本质上是错误的
语言模型参数。

在这里，我们
假设训练数据集是一个大型文本语料库，如
维基百科条目， [Project Gutenberg](https://en.wikipedia.org/wiki/Project_Gutenberg),
以及所有张贴在
网站上
单词的概率可以从相对单词中计算出来
训练数据集中给定单词的频率。
例如，估算值 $\hat{P}(\text{deep})$ can be calculated as the
任何以单词“deep”开头的句子的概率。A.

稍微不太准确的方法是计算所有发生的

“deep”一词，并将其除以

$$\hat{P}(\text{learning} \mid \text{deep}) = \frac{n(\text{deep, learning})}{n(\text{deep})},$$

其中， $n(x)$ 和 $n(x, x')$ 是单例出现的次数
和连续词对。
不幸的是，估计
由于
“深度学习”的出现要少得多。在里面
特别是，对于一些不寻常的单词组合，可能很难理解
找到足够的事件以获得准确的估计。
三个词的组合甚至更多，情况都会变得更糟。
将有许多看似合理的三个词组合，我们可能不会在数据集中看到。
除非我们提供一些解决方案来分配这样的单词组合非零计数，否则我们将无法在语言模型中使用它们。如果数据集很小，或者单词非常罕见，我们甚至可能找不到一个单词。

常用的策略是执行某种形式的*Laplace smoothing*。
解决办法是
将一个小常量添加到所有计数。
用 $n$ 表示表中的总字数
训练集
和 $m$ 唯一单词的数量。
此解决方案有助于解决单例问题，e.g., via

$$\begin{aligned}
	\hat{P}(x) & = \frac{n(x) + \epsilon_1/m}{n + \epsilon_1}, \\
	\hat{P}(x' \mid x) & = \frac{n(x, x') + \epsilon_2 \hat{P}(x')}{n(x) + \epsilon_2}, \\
	\hat{P}(x'' \mid x,x') & = \frac{n(x, x',x'') + \epsilon_3 \hat{P}(x'')}{n(x, x') + \epsilon_3}.
\end{aligned}$$

这里 $\epsilon_1,\epsilon_2$, 和 $\epsilon_3$ 是超参数。
以 $\epsilon_1$ 为例：
当 $\epsilon_1 = 0$ 时，不应用平滑；
当 $\epsilon_1$ 接近正无穷大时，
$\hat{P}(x)$ 接近均匀概率 $1/m$. 
以上是什么的一个相当原始的变体
其他技术也可以实现
引用：`Wood.Gasthaus.Archambeau.ea.2011`.


不幸的是，像这样的模型很快就会变得笨重
原因如下。首先，我们需要存储所有计数。
其次，这完全忽略了词语的含义。对于
例如，“猫”和“猫”应该出现在相关的上下文中。
很难根据其他情况调整此类模型，
然而，基于深度学习的语言模型非常适合于
考虑到这一点。
最后一句话
序列几乎肯定是新颖的，因此一个模型
统计以前看到的单词序列的频率在那里肯定会表现不佳。

## 马尔可夫模型与 $n$-grams

在讨论涉及深度学习的解决方案之前，我们需要更多的术语和概念。回想一下我们在：numref:`sec_sequence`.
让我们将此应用于语言建模。序列上的分布满足一阶马尔可夫性质，如果 $P(x_{t+1} \mid x_t, \ldots, x_1) = P(x_{t+1} \mid x_t)$. 高阶对应更长的依赖关系。这导致了我们可以应用于序列建模的许多近似值：

$$
\begin{aligned}
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2) P(x_3) P(x_4),\\
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_2) P(x_4  \mid  x_3),\\
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_1, x_2) P(x_4  \mid  x_2, x_3).
\end{aligned}
$$

涉及一个、两个和三个变量的概率公式通常分别称为*unigram*、*bigram*和*trigram*模型。在下面，我们将学习如何设计更好的模型。

## 自然语言统计

让我们看看这是如何在真实数据上工作的。
我们基于时间机器数据集构建了一个词汇表，如： :numref:`sec_text_preprocessing` 
中介绍的并打印出前10个最常用的单词。


In [ ]:
%load ../utils/djl-imports
%load ../utils/plot-utils
%load ../utils/PlotUtils.java

%load ../utils/Accumulator.java
%load ../utils/Animator.java
%load ../utils/Functions.java
%load ../utils/StopWatch.java
%load ../utils/Training.java
%load ../utils/timemachine/Vocab.java
%load ../utils/timemachine/RNNModelScratch.java
%load ../utils/timemachine/TimeMachine.java

In [ ]:
NDManager manager = NDManager.newBaseManager();

In [ ]:
String[][] tokens = TimeMachine.tokenize(TimeMachine.readTimeMachine(), "word");
// 由于每一行文字不一定是一个句子或段落，我们
// 连接所有文本行
List<String> corpus = new ArrayList<>();
for (int i = 0; i < tokens.length; i++) {
    for (int j = 0; j < tokens[i].length; j++) {
        if (tokens[i][j] != "") {
            corpus.add(tokens[i][j]);
        }
    }
}

Vocab vocab = new Vocab(new String[][] {corpus.toArray(new String[0])}, -1, new String[0]);
for (int i = 0; i < 10; i++) {
    Map.Entry<String, Integer> token = vocab.tokenFreqs.get(i);
    System.out.println(token.getKey() + ": " + token.getValue());   
}

正如我们所看到的，最流行的词实际上很无聊。
它们通常被称为“停止词”，因此被过滤掉。
尽管如此，它们仍然具有意义，我们仍将使用它们。
此外，很明显，词频衰减得相当快。$10^{\mathrm{th}}$ 最常用的单词少于 $1/5$ 最常用的单词。为了更好地理解，我们绘制了单词频率的图形。


In [ ]:
int n = vocab.tokenFreqs.size();
double[] freqs = new double[n];
double[] x = new double[n];
for (int i = 0; i < n; i++) {
    freqs[i] = (double) vocab.tokenFreqs.get(i).getValue();
    x[i] = (double) i;
}

PlotUtils.plotLogScale(new double[][] {x}, new double[][] {freqs}, new String[] {""},
                       "token: x", "frequency: n(x)");

我们在这里看到了一些非常基本的东西：单词频率以一种定义明确的方式迅速衰减。
将前几个单词作为例外处理后，所有剩余的单词大致沿着对数图上的一条直线。这意味着单词符合 *Zipf's law* 定律，
其中指出 $n_i$ $i^\mathrm{th}$ 最频繁单词的频率
是：

$$n_i \propto \frac{1}{i^\alpha},$$
:eqlabel:`eq_zipf_law`

这相当于

$$\log n_i = -\alpha \log i + c,$$

其中，$\alpha$ 是表是分布的指数，和 $c$ 是常数。
如果我们想通过计数统计和平滑来建模单词，这应该已经让我们暂停了。
毕竟，我们会明显高估尾部的频率，也就是不常出现的单词。但是其他的单词组合呢，比如bigrams，trigrams，等等？
让我们看看二元图频率的行为方式是否与单元图频率相同。


In [ ]:
String[] bigramTokens = new String[corpus.size()-1];
for (int i = 0; i < bigramTokens.length; i++) {
    bigramTokens[i] = corpus.get(i) + " " + corpus.get(i+1);
}
Vocab bigramVocab = new Vocab(new String[][] {bigramTokens}, -1, new String[0]);
for (int i = 0; i < 10; i++) {
    Map.Entry<String, Integer> token = bigramVocab.tokenFreqs.get(i);
    System.out.println(token.getKey() + ": " + token.getValue()); 
}

这里有一点值得注意。在十个最常见的词对中，有九个由两个停止词组成，只有一个与实际的书——《时间》有关。此外，让我们看看三角形频率是否以相同的方式运行。


In [ ]:
String[] trigramTokens = new String[corpus.size()-2];
for (int i = 0; i < trigramTokens.length; i++) {
    trigramTokens[i] = corpus.get(i) + " " + corpus.get(i+1) + " " + corpus.get(i+2);
}
Vocab trigramVocab = new Vocab(new String[][] {trigramTokens}, -1, new String[0]);
for (int i = 0; i < 10; i++) {
    Map.Entry<String, Integer> token = trigramVocab.tokenFreqs.get(i);
    System.out.println(token.getKey() + ": " + token.getValue()); 
}

最后，让我们想象一下这三个模型中的标记频率：单图、双图和三元图。


In [ ]:
n = bigramVocab.tokenFreqs.size();
double[] bigramFreqs = new double[n];
double[] bigramX = new double[n];
for (int i = 0; i < n; i++) {
    bigramFreqs[i] = (double) bigramVocab.tokenFreqs.get(i).getValue();
    bigramX[i] = (double) i;
}

n = trigramVocab.tokenFreqs.size();
double[] trigramFreqs = new double[n];
double[] trigramX = new double[n];
for (int i = 0; i < n; i++) {
    trigramFreqs[i] = (double) trigramVocab.tokenFreqs.get(i).getValue();
    trigramX[i] = (double) i;
}

PlotUtils.plotLogScale(new double[][] {x, bigramX, trigramX}, new double[][] {freqs, bigramFreqs, trigramFreqs}, 
                       new String[] {"unigram", "bigram", "trigram"}, "token: x", "frequency: n(x)");

这个数字相当令人兴奋，原因有很多。首先，除了单字母单词外，单词序列似乎也遵循齐普夫定律，尽管指数 $\alpha$ in :eqref:`eq_zipf_law` 定律，这取决于序列长度。
第二，不同的 $n$-grams 的数量并没有那么大。这给了我们希望，语言中有相当多的结构。
第三，许多 $n$-grams 很少出现，这使得拉普拉斯平滑非常不适合于语言建模。相反，我们将使用基于深度学习的模型。


## 读取长序列数据

由于序列数据本质上是连续的，我们需要解决
处理它的问题。
我们是以一种相当特别的方式这样做的：numref:`sec_sequence`.
当序列太长而无法由模型处理时
突然，
我们可能希望拆分这些序列以便阅读。
现在让我们描述一下一般策略。
在介绍模型之前，
假设我们将使用神经网络来训练语言模型，
其中，网络一次处理一小批具有预定义长度的序列，例如 $n$ 时间步长。
现在的问题是如何随机读取功能和标签的小批量。

首先，
由于文本序列可以任意长，
比如整本 *The Time Machine* 的书，
我们可以把这么长的序列分成子序列
使用相同的时间步数。
在训练我们的神经网络时，
这种子序列的小批量
将被输入到模型中。
假设网络处理一个子序列
of $n$ 时间步数
一次。
:numref:`fig_timemachine_5gram`
显示获取子序列的所有不同方式
来自原始文本序列，其中 $n=5$ 每个时间步的标记对应一个字符。
请注意，我们有相当大的自由度，因为我们可以选择一个指示初始位置的任意偏移量。

![拆分文本时，不同的偏移会导致不同的子序列。](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/timemachine-5gram.svg)
:label:`fig_timemachine_5gram`

事实上，它们都同样好。
但是，如果我们只选择一个偏移，
所有可能的子序列的覆盖范围有限
培训我们的网络。
因此
我们可以从一个随机偏移量开始划分一个序列
要同时获得*覆盖率*和*随机性*。
在下文中，
我们描述了如何实现这两个目标
*随机抽样*和*顺序分区*策略。

### 随机抽样

在随机抽样中，每个示例都是在原始长序列上任意捕获的子序列。
来自两个相邻随机小批量的子序列
迭代期间
在原始序列上不一定相邻。
对于语言建模，
目标是根据我们到目前为止看到的标记预测下一个标记，因此标签是原始序列，移动一个标记。


下面的代码每次都从数据中随机生成一个小批量。
这里，参数`batchSize`指定每个小批量中的子序列示例数
 `numSteps` 是预定义的时间步数
在每个子序列中。


In [ ]:
/**
 * 使用随机抽样生成一小批 子序列。
 */
public ArrayList<NDList>
        seqDataIterRandom(List<Integer> corpus, int batchSize, int numSteps, NDManager manager) {
    // 从一个随机偏移量（包括'numSteps-1'）开始到分区a
    // 序列
    corpus = corpus.subList(new Random().nextInt(numSteps - 1), corpus.size());
    // 减去1，因为我们需要考虑标签
    int numSubseqs = (corpus.size() - 1) / numSteps;
    // `numSteps` 长度子序列的起始指数
    List<Integer> initialIndices = new ArrayList<>();
    for (int i = 0; i < numSubseqs * numSteps; i += numSteps) {
        initialIndices.add(i);
    }
    // 在随机抽样中，来自两个相邻随机序列的子序列
    // 迭代过程中的小批量不一定在
    // 原始序列
    Collections.shuffle(initialIndices);

    int numBatches = numSubseqs / batchSize; 
        
    ArrayList<NDList> pairs = new ArrayList<NDList>();
    for (int i = 0; i < batchSize * numBatches; i += batchSize) {
        // 这里，`initialIndices` 包含的是
        // 子序列
        List<Integer> initialIndicesPerBatch = initialIndices.subList(i, i + batchSize);

        NDArray xNDArray = manager.create(new Shape(initialIndices.size(), numSteps), DataType.INT32);
        NDArray yNDArray = manager.create(new Shape(initialIndices.size(), numSteps), DataType.INT32);
        for (int j = 0; j < initialIndices.size(); j++) {
            ArrayList<Integer> X = data(initialIndices.get(j), corpus, numSteps);
            xNDArray.set(new NDIndex(j), manager.create(X.stream().mapToInt(Integer::intValue).toArray()));
            ArrayList<Integer> Y = data(initialIndices.get(j)+1, corpus, numSteps);
            yNDArray.set(new NDIndex(j), manager.create(Y.stream().mapToInt(Integer::intValue).toArray()));
        }
        NDList pair = new NDList();
        pair.add(xNDArray);
        pair.add(yNDArray);
        pairs.add(pair);
    }
    return pairs;
}

ArrayList<Integer> data(int pos, List<Integer> corpus, int numSteps) {
    // 返回从`pos` 开始的长度为`numSteps` 的序列
    return new ArrayList<Integer>(corpus.subList(pos, pos + numSteps));
}

让我们手动生成一个从0到34的序列。
我们假设
批次大小和时间步数分别为2和5，
这分别
意味着我们可以生成 $\lfloor (35 - 1) / 5 \rfloor= 6$ 特征标签子序列对。当小批量为2时，我们只能得到3个小批量。


In [ ]:
List<Integer> mySeq = new ArrayList<>();
for (int i = 0; i < 35; i++) {
    mySeq.add(i);
}

for (NDList pair : seqDataIterRandom(mySeq, 2, 5, manager)) {
    System.out.println("X:\n" + pair.get(0).toDebugString(50, 50, 50, 50));
    System.out.println("Y:\n" + pair.get(1).toDebugString(50, 50, 50, 50));
}

### 顺序分区

除了对原始序列进行随机抽样外，我们还可以确保
来自两个相邻小批次的子序列
迭代期间
在原始序列上相邻。
当在小批量上迭代时，此策略保持分割子序列的顺序，因此称为顺序分区。


In [ ]:
/**
 * 使用顺序分区生成一小批 子序列。
 */
public ArrayList<NDList> seqDataIterSequential(List<Integer> corpus, int batchSize, int numSteps, 
                                               NDManager manager) {
    // 从随机偏移量开始划分序列
    int offset = new Random().nextInt(numSteps);
    int numTokens = ((corpus.size() - offset - 1) / batchSize) * batchSize;
    
    NDArray Xs = manager.create(
        corpus.subList(offset, offset + numTokens).stream().mapToInt(Integer::intValue).toArray());
    NDArray Ys = manager.create(
        corpus.subList(offset + 1, offset + 1 + numTokens).stream().mapToInt(Integer::intValue).toArray());
    Xs = Xs.reshape(new Shape(batchSize, -1));
    Ys = Ys.reshape(new Shape(batchSize, -1));
    int numBatches = (int) Xs.getShape().get(1) / numSteps;
    
    
    ArrayList<NDList> pairs = new ArrayList<NDList>();
    for (int i = 0; i < numSteps * numBatches; i += numSteps) {
        NDArray X = Xs.get(new NDIndex(":, {}:{}", i, i + numSteps));
        NDArray Y = Ys.get(new NDIndex(":, {}:{}", i, i + numSteps));
        NDList pair = new NDList();
        pair.add(X);
        pair.add(Y);
        pairs.add(pair);
    }
    return pairs;
}

使用相同的设置，
让我们为通过顺序分区读取的每个小批量子序列打印特征`X` 和标签 `Y`
注意
来自两个相邻小批次的子序列
迭代期间
在原始序列上确实是相邻的。

In [ ]:
for (NDList pair : seqDataIterSequential(mySeq, 2, 5, manager)) {
    System.out.println("X:\n" + pair.get(0).toDebugString(10, 10, 10, 10));
    System.out.println("Y:\n" + pair.get(1).toDebugString(10, 10, 10, 10));
}

现在，我们将上述两个采样函数封装到一个类中，以便以后可以将其用作数据迭代器。


In [ ]:
public class SeqDataLoader implements Iterable<NDList> {
    public ArrayList<NDList> dataIter;
    public List<Integer> corpus;
    public Vocab vocab;
    public int batchSize;
    public int numSteps;
    
    /** 
     * 加载序列数据的迭代器
     */
    @SuppressWarnings("unchecked")
    public SeqDataLoader(int batchSize, int numSteps, boolean useRandomIter, int maxTokens) throws IOException, Exception {
        Pair<List<Integer>, Vocab> corpusVocabPair = TimeMachine.loadCorpusTimeMachine(maxTokens);
        this.corpus = corpusVocabPair.getKey();
        this.vocab = corpusVocabPair.getValue();
        
        this.batchSize = batchSize;
        this.numSteps = numSteps;
        if (useRandomIter) {
            dataIter = seqDataIterRandom(corpus, batchSize, numSteps, manager);
        }else {
            dataIter = seqDataIterSequential(corpus, batchSize, numSteps, manager);
        }
    }
    
    @Override
    public Iterator<NDList> iterator() {
        return dataIter.iterator();
    }
}

最后，我们定义了一个函数 `loadDataTimeMachine` ，它返回数据迭代器和词汇表。

In [ ]:
/**
 * 返回时间机器数据集的迭代器和词汇表
 */
public Pair<ArrayList<NDList>, Vocab> loadDataTimeMachine(int batchSize, int numSteps, boolean useRandomIter, int maxTokens) throws IOException, Exception {
    SeqDataLoader seqData = new SeqDataLoader(batchSize, numSteps, useRandomIter, maxTokens);
    return new Pair(seqData.dataIter, seqData.vocab); // ArrayList<NDList>, Vocab
}

## 总结

* 语言模型是自然语言处理的关键。
* $n$-grams 通过截断依赖关系为处理长序列提供了一个方便的模型。
* 长序列存在这样一个问题：它们很少发生或从未发生过。
* 齐普夫定律不仅支配单格词的分布，也支配其他 $n$-grams 词的分布。
* 存在大量的结构，但没有足够的频率通过拉普拉斯平滑有效地处理不频繁的单词组合。
* 读取长序列的主要选择是随机采样和顺序分区。后者可以确保迭代期间来自两个相邻小批量的子序列在原始序列上相邻。


## 练习

1. 假设训练数据集中有$100,000$单词。四克需要存储多少字频率和多字相邻频率？
2. 你将如何模拟对话？
3. 估计单图、双图和三元图的齐普夫定律指数。
4. 对于读取长序列数据，您还能想到哪些其他方法？
5. 考虑我们用来读取长序列的随机偏移量。
    1. 为什么有一个随机偏移是个好主意？
    2. 它真的会导致文档序列上的完全均匀分布吗？
    3. 你要怎么做才能使事情变得更加统一？
6. 如果我们希望序列示例是一个完整的句子，那么在小批量采样中会引入什么样的问题？我们如何解决这个问题？